In [2]:
import keras
import pandas as pd
import os
import tensorflow as tf
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Sequential, Model
from keras.layers import Dense, InputLayer, Input
import keras.backend as K

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split


def custom_loss(layer):
    def loss(y_true,y_pred):
        return K.mean(K.mean(K.square(y_pred - y_true)) + K.square(layer), axis=-1)
    return loss

model = Sequential()
model.add(Dense(3))
model.add(Dense(16, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(4))

# inputs = Input(shape=(3,))
# layer1 = Dense(16, activation='relu')(inputs)
# layer2 = Dense(32, activation='relu')(layer1)
# predictions = Dense(4)(layer2)
# model = Model(inputs=inputs, outputs=predictions)

# f = custom_loss(inputs)
model.compile(loss='mse', optimizer='SGD', metrics=['mse'])


Using TensorFlow backend.


In [5]:
dir_path = '../../../experiments_prokhn/controllers/mined'
data = []
for file_name in [i for i in os.listdir(dir_path) if '.csv' in i]:
    _data = pd.read_csv(os.path.join(dir_path, file_name), sep=';', engine='python')
    _data.columns = ['delta1', 'delta2', 'delta3', 'v1', 'v2', 'v3', 'v4']
    data.append(_data)
data = pd.concat(data, axis=0)

columns = [column for column in data.columns if 'delta' in column]
x = data[columns].values
y = data.drop(columns, axis=1).values
model.fit(x, y, batch_size=64, epochs=1, validation_split=0.2, shuffle=True, 
              callbacks=[ModelCheckpoint('model.hd5', save_best_only=True), 
                         EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=20)])
# print(y)
# print(x)

Train on 300869 samples, validate on 75218 samples
Epoch 1/1
300869/300869 [==============================] - 9s 30us/step - loss: 0.1028 - mean_squared_error: 0.1028 - val_loss: 0.1051 - val_mean_squared_error: 0.1051


In [15]:
import numpy as np
model.predict(np.array([[1, 1, 1]]))

array([[-0.49023592, -0.7977226 ,  0.57501256, -0.05241978]],
      dtype=float32)

In [12]:
print(x)

[[ 5.77928356 -0.35792363 -4.69759168]
 [ 5.77928356 -0.35792363 -4.69759168]
 [ 5.77928356 -0.35792354 -4.66568829]
 ...
 [-0.15792131 -0.07633683 -0.03132203]
 [-0.23942485 -0.07045121 -0.13129343]
 [-0.31451967 -0.06069233 -0.14728781]]
